In [ ]:
from lsst.daf.butler import Butler

In [ ]:
# Change the output collection to match your ticket, e.g. wtg/tickets/PIPE2D-1234
OUTPUT_COLLECTION = 'wtg/example'

In [ ]:
# These probably dont' need to change.
DATASTORE = '/work/datastore/'
COLLECTIONS = 'PFS/default'
INSTRUMENT = 'lsst.obs.pfs.PrimeFocusSpectrograph'

# These are the pipelines that we are running.
REDUCE_PIPELINE_PATH = '$DRP_STELLA_DIR/pipelines/reduceExposure.yaml'
QA_PIPELINE_PATH = f'/work/wtg/drp_qa_dev/pipelines/detectorMapQa.yaml'

# Pipeline run options
LOG_FILE = './pipetask.log'
NUM_JOBS = 25

## Run `reduceExposure` pipeline

This will generate the `lines` and adjusted `detectorMap` objects we use for the DetectorMap QA.

We use the `!` operator to call the `pipetask` command the same as we would in a shell. 

Here we are just processing a simple set of calibration exposures, a Quartz and one of each lamp line, which we will define in our data_select string.

In [ ]:
data_select = "exposure in (114641, 114644, 114647, 114651, 114655, 114659) and arm in ('b', 'r', 'n')"

In [ ]:
!pipetask \
    --long-log \
    --no-log-tty \
    --log-level "pfs.drp.stella=INFO" \
    --log-file {LOG_FILE} \
    run \
    -b {DATASTORE} \
    -j {NUM_JOBS} \
    --instrument {INSTRUMENT} \
    -i {COLLECTIONS} \
    -o {OUTPUT_COLLECTION} \
    -p {REDUCE_PIPELINE_PATH} \
    -d "{data_select}"

## Run `detectorMapQa` pipeline

The `detectorMapQa` runs a number of separate QA related tasks.

We specify a few options to show the syntax but the default options should be okay for most processing.

In [ ]:
use_sigma = False
spatial_range = 0.1
wavelength_range = 0.1

In [ ]:
!pipetask \
    --long-log \
    --no-log-tty \
    --log-file {LOG_FILE} \
    run \
    --register-dataset-types \
    -b {DATASTORE} \
    -j {NUM_JOBS} \
    --instrument {INSTRUMENT} \
    -i {COLLECTIONS} \
    -o {OUTPUT_COLLECTION} \
    -p "{QA_PIPELINE_PATH}" \
    -d "{data_select}" \
    -c dmResiduals:useSigmaRange={use_sigma} \
    -c dmResiduals:spatialRange={spatial_range} \
    -c dmResiduals:wavelengthRange={wavelength_range} \
    --fail-fast

## Get stored objects from the butler

We use the `OUTPUT_COLLECTION` to look up the objects stored by the tasks.

In [ ]:
butler = Butler(DATASTORE, collections=OUTPUT_COLLECTION)


The QA task for individual exposures produces a plot (saved as a `.png` image)
and a `pandas.DataFrame` that has two rows of statistics, one for the `USED`
points and one for the `RESERVED` points.

The `USED` points come from the adjustment itself and the `RESERVED` points are
 what we want to do the QA checks on.

The statistics are stored under the `dmQaResidualStats` and the plot is `dmQaResidualPlot`. These both require the `(arm, spectrograph, exposure)` dimensions, which we define here as a traditional DataId.

In [ ]:
data_id = dict(arm='r', spectrograph=2, exposure=114641)

### dmQaResidualStats

Statistics for an individual detector for a given exposure.

In [ ]:
# Show the RESERVED and USED stats for the given detector exposure.
butler.get('dmQaResidualStats', data_id).T

### dmQaResidualPlot

The 2D residaul plot of an indvidual detector for a given exposure.

The residual plots are stored as `png` images and the butler can only return a path to those objects, but we can display those in a notebook using the path.

In [ ]:
butler.getURI('dmQaResidualPlot', data_id)

In [ ]:
# Use the .unquoted_path property.
from IPython.display import Image
Image(filename=butler.getURI('dmQaResidualPlot', data_id).unquoted_path)

### dmQaDetectorStats

Aggregate detector stats for all decector and exposure combinations in the collection.

In [ ]:
butler.get('dmQaDetectorStats').head()

### dmQaCombinedResidualPlot

A multi-page pdf report that shows aggregate plot statistics based on the `dmQaDetectorStats` data above.

Like images, the butler cannot directly display a PDF but only return a path. We can copy that file from the datastore to the local directory to look at it.

In [ ]:
butler.getURI('dmQaCombinedResidualPlot')

In [ ]:
# Copy the file to our local directory
!cp {butler.getURI('dmQaCombinedResidualPlot').unquoted_path} .